In [13]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from fuzzywuzzy import fuzz, process
import re
from collections import defaultdict
import warnings
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl

warnings.filterwarnings('ignore')



## Основной класс

In [14]:
class ColumnComparator:
    def __init__(self, similarity_threshold=75, data_similarity_threshold=0.7):
        self.similarity_threshold = similarity_threshold
        self.data_similarity_threshold = data_similarity_threshold
        self.column_profiles = {}
        
    def preprocess_column_name(self, name):
        """Предобработка названия колонки"""
        if pd.isna(name):
            return ""
        name = str(name).lower().strip()
        # Удаляем специальные символы и лишние пробелы
        name = re.sub(r'[^\w\s]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        return name.strip()
    
    def convert_to_string(self, value):
        """Конвертирует значение в строку с обработкой NaN"""
        if pd.isna(value):
            return ""
        return str(value).strip()
    
    def parse_date(self, date_str):
        """Пытается распарсить дату из строки в различных форматах"""
        if pd.isna(date_str) or date_str == '':
            return None
            
        date_str = str(date_str).strip()
        
        # Список возможных форматов дат
        date_formats = [
            '%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y', '%m/%d/%Y',
            '%Y.%m.%d', '%Y/%m/%d', '%d-%m-%Y', '%m-%d-%Y',
            '%Y-%m-%d %H:%M:%S', '%d.%m.%Y %H:%M:%S', '%d/%m/%Y %H:%M:%S',
            '%Y.%m.%d %H:%M:%S', '%Y/%m/%d %H:%M:%S', '%d-%m-%Y %H:%M:%S',
            '%Y-%m-%d %H:%M', '%d.%m.%Y %H:%M', '%d/%m/%Y %H:%M',
            '%Y.%m.%d %H:%M', '%Y/%m/%d %H:%M', '%d-%m-%Y %H:%M'
        ]
        
        for fmt in date_formats:
            try:
                return pd.to_datetime(date_str, format=fmt)
            except (ValueError, TypeError):
                continue
        
        # Если ни один формат не подошел, пробуем автоматическое определение
        try:
            return pd.to_datetime(date_str, infer_datetime_format=True)
        except (ValueError, TypeError):
            return None
    
    def are_values_equal(self, val1, val2):
        """Умное сравнение значений с учетом числовых типов и дат"""
        # Оба значения NaN
        if pd.isna(val1) and pd.isna(val2):
            return True
        
        # Один NaN, другой нет
        if pd.isna(val1) or pd.isna(val2):
            return False
        
        # Пробуем сравнить как даты
        date1 = self.parse_date(val1)
        date2 = self.parse_date(val2)
        
        if date1 is not None and date2 is not None:
            # Сравниваем даты (игнорируем время если оно не указано в обоих)
            if date1 == date2:
                return True
            # Проверяем, может быть это одна и та же дата в разных форматах
            if date1.date() == date2.date():
                return True
            return False
        
        # Если оба значения не даты, пробуем сравнить как числа
        try:
            num1 = float(val1)
            num2 = float(val2)
            # Сравниваем числа с небольшой погрешностью
            return abs(num1 - num2) < 0.000001
        except (ValueError, TypeError):
            # Если не числа, сравниваем как строки
            str1 = str(val1).strip()
            str2 = str(val2).strip()
            return str1 == str2
    
    def get_column_profile(self, df, column_name):
        """Создает профиль для колонки с метаданными и характеристиками"""
        # Конвертируем все значения в строку для сравнения
        string_series = df[column_name].apply(self.convert_to_string)
        original_series = df[column_name].dropna()
        
        profile = {
            'name': column_name,
            'preprocessed_name': self.preprocess_column_name(column_name),
            'dtype': str(df[column_name].dtype),
            'total_count': len(df),
            'non_null_count': len(original_series),
            'null_percentage': (df[column_name].isnull().sum() / len(df)) * 100,
            'unique_count': string_series.nunique(),
            'unique_percentage': (string_series.nunique() / len(string_series)) * 100 if len(string_series) > 0 else 0,
            'sample_values': string_series.head(5).tolist(),
            'value_lengths': string_series.str.len().describe().to_dict() if len(string_series) > 0 else {}
        }
        
        # Определяем тип данных более точно
        if pd.api.types.is_numeric_dtype(original_series):
            profile['data_type'] = 'numeric'
            profile['stats'] = original_series.describe().to_dict() if len(original_series) > 0 else {}
        elif pd.api.types.is_datetime64_any_dtype(original_series):
            profile['data_type'] = 'datetime'
        else:
            profile['data_type'] = 'string'
            # Анализ паттернов для строковых данных
            if len(string_series) > 0:
                sample_str = string_series.iloc[0]
                if re.match(r'^[A-Za-z]{2,3}-\d{4,6}$', sample_str):
                    profile['pattern'] = 'code_pattern'
                elif re.match(r'^\d{2}\.\d{2}\.\d{4}$', sample_str) or \
                     re.match(r'^\d{4}-\d{2}-\d{2}$', sample_str) or \
                     re.match(r'^\d{2}/\d{2}/\d{4}$', sample_str):
                    profile['pattern'] = 'date_pattern'
                elif '@' in sample_str:
                    profile['pattern'] = 'email_pattern'
        
        return profile
    
    def calculate_name_similarity(self, name1, name2):
        """Вычисляет схожесть названий колонок"""
        return fuzz.token_sort_ratio(name1, name2)
    
    def calculate_data_similarity(self, profile1, profile2):
        """Вычисляет схожесть данных в колонках"""
        similarity_score = 0
        factors_checked = 0
        
        # Сравнение типов данных (ослабляем требования из-за конвертации в строку)
        if profile1['data_type'] == profile2['data_type']:
            similarity_score += 20
        elif (profile1['data_type'] in ['numeric', 'datetime'] and 
              profile2['data_type'] in ['numeric', 'datetime']):
            similarity_score += 15  # Частичное совпадение для числовых/дата типов
        factors_checked += 20
        
        # Сравнение уникальности
        uniqueness_diff = abs(profile1['unique_percentage'] - profile2['unique_percentage'])
        if uniqueness_diff < 20:  # Разница менее 20%
            similarity_score += 25
        factors_checked += 25
        
        # Сравнение распределения длин строк
        if profile1['value_lengths'] and profile2['value_lengths']:
            len_diff = abs(profile1['value_lengths'].get('mean', 0) - profile2['value_lengths'].get('mean', 0))
            if len_diff < 3:  # Средняя длина отличается менее чем на 3 символа
                similarity_score += 25
            factors_checked += 25
        
        # Сравнение паттернов
        if profile1.get('pattern') == profile2.get('pattern'):
            similarity_score += 30
            factors_checked += 30
        
        return (similarity_score / factors_checked) * 100 if factors_checked > 0 else 0
    
    def find_column_matches(self, df1, df2, predefined_key_columns=None):
        """Находит соответствия между колонками двух DataFrame"""
        matches = []
        
        # Конвертируем все данные в строки для единообразного сравнения
        df1_str = df1.applymap(self.convert_to_string)
        df2_str = df2.applymap(self.convert_to_string)
        
        # Создаем профили для всех колонок
        profiles1 = {col: self.get_column_profile(df1_str, col) for col in df1_str.columns}
        profiles2 = {col: self.get_column_profile(df2_str, col) for col in df2_str.columns}
        
        # Если заданы predefined ключевые колонки, добавляем их первыми
        if predefined_key_columns:
            key_col_a, key_col_b = predefined_key_columns
            if key_col_a in df1_str.columns and key_col_b in df2_str.columns:
                matches.append({
                    'system_a_column': key_col_a,
                    'system_b_column': key_col_b,
                    'name_similarity': 100,  # Максимальная схожесть для predefined
                    'data_similarity': 100,
                    'combined_score': 100,
                    'status': 'PREDEFINED_KEY'
                })
                print(f"Добавлена predefined ключевая колонка: {key_col_a} -> {key_col_b}")
        
        for col1, profile1 in profiles1.items():
            # Пропускаем если уже добавлена как predefined
            if any(match['system_a_column'] == col1 for match in matches):
                continue
                
            best_match = None
            best_score = 0
            
            for col2, profile2 in profiles2.items():
                # Пропускаем если уже добавлена как predefined
                if any(match['system_b_column'] == col2 for match in matches):
                    continue
                
                # Схожесть по имени
                name_similarity = self.calculate_name_similarity(
                    profile1['preprocessed_name'], 
                    profile2['preprocessed_name']
                )
                
                # Схожесть по данным
                data_similarity = self.calculate_data_similarity(profile1, profile2)
                
                # Комбинированная оценка (40% имя, 60% данные)
                combined_score = (name_similarity * 0.4) + (data_similarity * 0.6)
                
                if combined_score > best_score:
                    best_score = combined_score
                    best_match = (col2, name_similarity, data_similarity, combined_score)
            
            if best_match and best_score >= self.similarity_threshold:
                matches.append({
                    'system_a_column': col1,
                    'system_b_column': best_match[0],
                    'name_similarity': best_match[1],
                    'data_similarity': best_match[2],
                    'combined_score': best_match[3],
                    'status': 'MATCHED'
                })
            else:
                matches.append({
                    'system_a_column': col1,
                    'system_b_column': 'NO_MATCH',
                    'name_similarity': 0,
                    'data_similarity': 0,
                    'combined_score': 0,
                    'status': 'UNMATCHED'
                })
        
        return pd.DataFrame(matches), profiles1, profiles2


## Расширенный класс

In [15]:
class AdvancedColumnComparator(ColumnComparator):
    def __init__(self, similarity_threshold=75, data_similarity_threshold=0.7):
        super().__init__(similarity_threshold, data_similarity_threshold)
        self.mismatch_fill = PatternFill(start_color="FFE6E6", end_color="FFE6E6", fill_type="solid")
        self.header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
        self.summary_fill = PatternFill(start_color="DCE6F1", end_color="DCE6F1", fill_type="solid")
    
    def create_detailed_mismatch_report(self, df1, df2, matches_df, key_columns, folder_name, output_path):
        """Создает детальный отчет с расхождениями"""
        
        print("Начинаем создание отчета...")
        print(f"df1 колонки: {list(df1.columns)}")
        print(f"df2 колонки: {list(df2.columns)}")
        print(f"Key columns: {key_columns}")
        
        # Конвертируем все данные в строки для сравнения
        df1_str = df1.applymap(self.convert_to_string)
        df2_str = df2.applymap(self.convert_to_string)
        
        # Создаем Excel файл
        wb = Workbook()
        ws1 = wb.active
        ws1.title = "Детальные расхождения"
        
        # Получаем сопоставленные пары
        matched_pairs = matches_df[matches_df['status'].isin(['MATCHED', 'PREDEFINED_KEY'])]
        print(f"Сопоставленные пары: {len(matched_pairs)}")
        
        # Создаем merge по ключевым колонкам
        key_col_a, key_col_b = key_columns
        merged_df = pd.merge(df1_str, df2_str, left_on=key_col_a, right_on=key_col_b, 
                           how='outer', suffixes=('_A', '_B'), indicator=True)
        
        print(f"После merge: {len(merged_df)} строк")
        print(f"Колонки после merge: {list(merged_df.columns)}")
        
        # Подготавливаем заголовки для отчета
        headers = [
            f'KEY_A_{key_col_a}',
            f'KEY_B_{key_col_b}', 
            'Status'
        ]
        
        # Добавляем колонки для каждой сопоставленной пары
        column_mapping = {}
        for _, match in matched_pairs.iterrows():
            col_a = match['system_a_column']
            col_b = match['system_b_column']
            headers.extend([f'A_{col_a}', f'B_{col_b}', f'Diff_{col_a}'])
            column_mapping[col_a] = col_b
        
        headers.append('Mismatch_Count')
        
        # Записываем заголовки
        for col_idx, header in enumerate(headers, 1):
            cell = ws1.cell(row=1, column=col_idx, value=header)
            cell.fill = self.header_fill
            cell.font = Font(bold=True, color="FFFFFF")
            cell.alignment = Alignment(horizontal='center')
        
        # ЗАРАНЕЕ ПОДСЧИТАЕМ РАСХОЖДЕНИЯ ПО КОЛОНКАМ
        column_mismatch_stats = []
        for col_a, col_b in column_mapping.items():
            col_mismatches = 0
            for _, row in merged_df.iterrows():
                col_a_merged = f'{col_a}_A'
                col_b_merged = f'{col_b}_B'
                
                val_a = row.get(col_a_merged, '')
                val_b = row.get(col_b_merged, '')
                
                # Если значения не найдены, пробуем найти в оригинальных колонках
                if val_a == '' and col_a in row:
                    val_a = row[col_a]
                if val_b == '' and col_b in row:
                    val_b = row[col_b]
                
                # Все значения уже конвертированы в строки, используем умное сравнение
                if not self.are_values_equal(val_a, val_b):
                    col_mismatches += 1
            
            column_mismatch_stats.append({
                'system_a_column': col_a,
                'system_b_column': col_b,
                'mismatch_count': col_mismatches,
                'total_comparisons': len(merged_df),
                'mismatch_percentage': (col_mismatches / len(merged_df) * 100) if len(merged_df) > 0 else 0
            })
        
        # Заполняем данные в лист
        row_idx = 2
        all_mismatches_count = 0
        mismatch_rows_indices = []
        
        for idx, merged_row in merged_df.iterrows():
            row_data = {}
            mismatch_count = 0
            
            # Ключевые колонки
            key_a_val = merged_row.get(f'{key_col_a}_A', merged_row.get(key_col_a, ''))
            key_b_val = merged_row.get(f'{key_col_b}_B', merged_row.get(key_col_b, ''))
            
            row_data[f'KEY_A_{key_col_a}'] = key_a_val
            row_data[f'KEY_B_{key_col_b}'] = key_b_val
            row_data['Status'] = merged_row.get('_merge', 'both')
            
            # Данные по сопоставленным колонкам
            for col_a, col_b in column_mapping.items():
                col_a_merged = f'{col_a}_A'
                col_b_merged = f'{col_b}_B'
                
                # Получаем значения (уже конвертированные в строки)
                val_a = merged_row.get(col_a_merged, '')
                val_b = merged_row.get(col_b_merged, '')
                
                # Если значения не найдены, пробуем найти в оригинальных колонках
                if val_a == '' and col_a in merged_row:
                    val_a = merged_row[col_a]
                if val_b == '' and col_b in merged_row:
                    val_b = merged_row[col_b]
                
                # Используем умное сравнение
                is_mismatch = not self.are_values_equal(val_a, val_b)
                
                row_data[f'A_{col_a}'] = val_a
                row_data[f'B_{col_b}'] = val_b
                row_data[f'Diff_{col_a}'] = 'РАСХОЖДЕНИЕ' if is_mismatch else 'OK'
                
                if is_mismatch:
                    mismatch_count += 1
                    all_mismatches_count += 1
            
            row_data['Mismatch_Count'] = mismatch_count
            
            # Сохраняем информацию о строках с расхождениями
            if mismatch_count > 0:
                mismatch_rows_indices.append(idx)
            
            # Записываем строку в Excel
            for col_idx, header in enumerate(headers, 1):
                value = row_data.get(header, '')
                cell = ws1.cell(row=row_idx, column=col_idx, value=value)
                
                # Закрашиваем расхождения
                if 'РАСХОЖДЕНИЕ' in str(value):
                    cell.fill = self.mismatch_fill
                
                # Закрашиваем всю строку если есть расхождения
                elif header == 'Mismatch_Count' and value > 0:
                    for col in range(1, len(headers) + 1):
                        ws1.cell(row=row_idx, column=col).fill = self.mismatch_fill
            
            row_idx += 1
        
        # Настраиваем ширину колонок
        for column in ws1.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 30)
            ws1.column_dimensions[column_letter].width = adjusted_width
        
        # === ЛИСТ 2: Сводная статистика ===
        ws2 = wb.create_sheet("Сводная статистика")
        
        # Расчет статистики
        total_rows = len(merged_df)
        rows_with_mismatches = len(mismatch_rows_indices)
        rows_without_mismatches = total_rows - rows_with_mismatches
        
        total_columns_a = len(df1.columns)
        total_columns_b = len(df2.columns)
        matched_columns = len(matched_pairs)
        
        # Подсчет несопоставленных колонок
        unmatched_columns_a = []
        unmatched_columns_b = []
        
        for _, match_row in matches_df.iterrows():
            status = match_row['status']
            system_a_col = match_row['system_a_column']
            system_b_col = match_row['system_b_column']
            
            if status == 'UNMATCHED' and system_a_col != 'NO_MATCH':
                unmatched_columns_a.append(system_a_col)
            elif status == 'UNMATCHED_B':
                unmatched_columns_b.append(system_b_col)
        
        # Статистика по merge
        merge_stats = merged_df['_merge'].value_counts()
        left_only = merge_stats.get('left_only', 0)
        right_only = merge_stats.get('right_only', 0)
        both_sides = merge_stats.get('both', 0)
        
        # Расчет процента совпадения
        total_comparisons = total_rows * matched_columns
        
        if total_comparisons > 0:
            overall_match_percentage = 100 - (all_mismatches_count / total_comparisons * 100)
        else:
            overall_match_percentage = 0
        
        # ФИЛЬТРУЕМ КОЛОНКИ С РАСХОЖДЕНИЯМИ
        columns_with_mismatches = [col for col in column_mismatch_stats if col['mismatch_count'] > 0]
        
        summary_data = [
            ["ПАРАМЕТР", "ЗНАЧЕНИЕ"],
            ["Общая информация", ""],
            ["Название папки", folder_name],
            ["Дата анализа", pd.Timestamp.now().strftime("%Y-%m-%d %H:%M")],
            ["", ""],
            ["Статистика по строкам", ""],
            ["Всего строк", total_rows],
            ["Строк без расхождений", rows_without_mismatches],
            ["Строк с расхождениями", rows_with_mismatches],
            ["Процент строк с расхождениями", f"{(rows_with_mismatches/total_rows)*100:.2f}%" if total_rows > 0 else "0%"],
            ["Строк только в системе A", left_only],
            ["Строк только в системе B", right_only],
            ["Строк в обеих системах", both_sides],
            ["", ""],
            ["Статистика по колонкам", ""],
            ["Всего колонок в системе A", total_columns_a],
            ["Всего колонок в системе B", total_columns_b],
            ["Сопоставленных колонок", matched_columns],
            ["Несопоставленных в системе A", len(unmatched_columns_a)],
            ["Несопоставленных в системе B", len(unmatched_columns_b)],
            ["Процент сопоставленных", f"{(matched_columns/min(total_columns_a, total_columns_b))*100:.2f}%" if min(total_columns_a, total_columns_b) > 0 else "0%"],
            ["Колонок с расхождениями", len(columns_with_mismatches)],
            ["", ""],
            ["Статистика по расхождениям", ""],
            ["Всего проверок значений", total_comparisons],
            ["Всего расхождений", all_mismatches_count],
            ["Общий процент совпадения", f"{overall_match_percentage:.2f}%"],
            ["Среднее расхождений на строку", f"{(all_mismatches_count/total_rows):.2f}" if total_rows > 0 else "0"]
        ]
        
        # Записываем сводную статистику
        current_row = 1
        for row_data in summary_data:
            for col_idx, value in enumerate(row_data, 1):
                cell = ws2.cell(row=current_row, column=col_idx, value=value)
                if current_row == 1:
                    cell.fill = self.header_fill
                    cell.font = Font(bold=True, color="FFFFFF")
                    cell.alignment = Alignment(horizontal='center')
                elif any(x in str(value) for x in ["ПАРАМЕТР", "Общая информация", "Статистика по строкам", 
                                                 "Статистика по колонкам", "Статистика по расхождениям"]):
                    cell.fill = self.summary_fill
                    cell.font = Font(bold=True)
            current_row += 1
        
        # Статистика по колонкам с расхождениями
        if columns_with_mismatches:
            current_row += 1
            ws2.cell(row=current_row, column=1, value="Детали по колонкам с расхождениями").font = Font(bold=True)
            current_row += 1
            
            headers = ["Колонка A", "Колонка B", "Расхождения", "Всего проверок", "% Расхождений"]
            for col_idx, header in enumerate(headers, 1):
                cell = ws2.cell(row=current_row, column=col_idx, value=header)
                cell.fill = self.header_fill
                cell.font = Font(bold=True, color="FFFFFF")
                cell.alignment = Alignment(horizontal='center')
            current_row += 1
            
            for col_stats in columns_with_mismatches:
                ws2.cell(row=current_row, column=1, value=col_stats['system_a_column'])
                ws2.cell(row=current_row, column=2, value=col_stats['system_b_column'])
                ws2.cell(row=current_row, column=3, value=col_stats['mismatch_count'])
                ws2.cell(row=current_row, column=4, value=col_stats['total_comparisons'])
                ws2.cell(row=current_row, column=5, value=f"{col_stats['mismatch_percentage']:.2f}%")
                current_row += 1
        
        # Несопоставленные колонки из системы A
        if unmatched_columns_a:
            current_row += 2
            ws2.cell(row=current_row, column=1, value="Несопоставленные колонки из системы A").font = Font(bold=True)
            current_row += 1
            
            headers_a = ["Колонка", "Тип данных", "Кол-во строк", "Кол-во уникальных", "% Уникальных", "Примеры значений"]
            for col_idx, header in enumerate(headers_a, 1):
                cell = ws2.cell(row=current_row, column=col_idx, value=header)
                cell.fill = self.header_fill
                cell.font = Font(bold=True, color="FFFFFF")
                cell.alignment = Alignment(horizontal='center')
            current_row += 1
            
            for col_name in unmatched_columns_a:
                if col_name in df1.columns:
                    col_data = df1[col_name]
                    unique_count = col_data.nunique()
                    total_count = len(col_data)
                    unique_percentage = (unique_count / total_count * 100) if total_count > 0 else 0
                    
                    # Получаем примеры значений (первые 3 не-NaN значения)
                    sample_values = col_data.dropna().head(3).tolist()
                    sample_str = ", ".join([str(x) for x in sample_values[:2]])  # Берем первые 2 значения
                    if len(sample_values) > 2:
                        sample_str += ", ..."
                    
                    ws2.cell(row=current_row, column=1, value=col_name)
                    ws2.cell(row=current_row, column=2, value=str(col_data.dtype))
                    ws2.cell(row=current_row, column=3, value=total_count)
                    ws2.cell(row=current_row, column=4, value=unique_count)
                    ws2.cell(row=current_row, column=5, value=f"{unique_percentage:.1f}%")
                    ws2.cell(row=current_row, column=6, value=sample_str)
                    current_row += 1
        
        # Несопоставленные колонки из системы B
        if unmatched_columns_b:
            current_row += 2
            ws2.cell(row=current_row, column=1, value="Несопоставленные колонки из системы B").font = Font(bold=True)
            current_row += 1
            
            headers_b = ["Колонка", "Тип данных", "Кол-во строк", "Кол-во уникальных", "% Уникальных", "Примеры значений"]
            for col_idx, header in enumerate(headers_b, 1):
                cell = ws2.cell(row=current_row, column=col_idx, value=header)
                cell.fill = self.header_fill
                cell.font = Font(bold=True, color="FFFFFF")
                cell.alignment = Alignment(horizontal='center')
            current_row += 1
            
            for col_name in unmatched_columns_b:
                if col_name in df2.columns:
                    col_data = df2[col_name]
                    unique_count = col_data.nunique()
                    total_count = len(col_data)
                    unique_percentage = (unique_count / total_count * 100) if total_count > 0 else 0
                    
                    # Получаем примеры значений (первые 3 не-NaN значения)
                    sample_values = col_data.dropna().head(3).tolist()
                    sample_str = ", ".join([str(x) for x in sample_values[:2]])  # Берем первые 2 значения
                    if len(sample_values) > 2:
                        sample_str += ", ..."
                    
                    ws2.cell(row=current_row, column=1, value=col_name)
                    ws2.cell(row=current_row, column=2, value=str(col_data.dtype))
                    ws2.cell(row=current_row, column=3, value=total_count)
                    ws2.cell(row=current_row, column=4, value=unique_count)
                    ws2.cell(row=current_row, column=5, value=f"{unique_percentage:.1f}%")
                    ws2.cell(row=current_row, column=6, value=sample_str)
                    current_row += 1
        
        # Настраиваем ширину колонок для всех разделов
        column_widths = {
            1: 25,  # Колонка
            2: 15,  # Тип данных
            3: 12,  # Кол-во строк
            4: 15,  # Кол-во уникальных
            5: 15,  # % Уникальных
            6: 30   # Примеры значений
        }
        
        for col_idx, width in column_widths.items():
            ws2.column_dimensions[openpyxl.utils.get_column_letter(col_idx)].width = width
        
        # Сохраняем файл
        output_file = output_path / f"comparison_report_{folder_name}.xlsx"
        wb.save(output_file)
        
        print(f"Отчет сохранен: {output_file}")
        print(f"Всего строк в отчете: {total_rows}")
        print(f"Строк с расхождениями: {rows_with_mismatches}")
        print(f"Всего расхождений: {all_mismatches_count}")
        print(f"Колонок с расхождениями: {len(columns_with_mismatches)}")
        print(f"Несопоставленных колонок из системы A: {len(unmatched_columns_a)}")
        print(f"Несопоставленных колонок из системы B: {len(unmatched_columns_b)}")
        
        # Детальная отладка по колонкам
        print("\nДетальная статистика по колонкам:")
        for col_stats in column_mismatch_stats:
            if col_stats['mismatch_count'] > 0:
                print(f"  {col_stats['system_a_column']} -> {col_stats['system_b_column']}: {col_stats['mismatch_count']} расхождений")
        
        return {
            'folder_name': folder_name,
            'total_rows': total_rows,
            'rows_with_mismatches': rows_with_mismatches,
            'rows_without_mismatches': rows_without_mismatches,
            'total_columns_a': total_columns_a,
            'total_columns_b': total_columns_b,
            'matched_columns': matched_columns,
            'unmatched_columns_a': len(unmatched_columns_a),
            'unmatched_columns_b': len(unmatched_columns_b),
            'total_mismatches': all_mismatches_count,
            'match_percentage': overall_match_percentage,
            'left_only_rows': left_only,
            'right_only_rows': right_only,
            'both_sides_rows': both_sides,
            'columns_with_mismatches': len(columns_with_mismatches)
        }


## Доп функции

In [16]:
def analyze_folder_structure(base_path):
    """Анализирует структуру папок и находит пары файлов"""
    folder_pairs = []
    base_path = Path(base_path)
    
    for folder in base_path.iterdir():
        if folder.is_dir():
            excel_files = list(folder.glob('*.xlsx')) + list(folder.glob('*.xls'))
            if len(excel_files) == 2:
                folder_pairs.append({
                    'folder': folder.name,
                    'file1': excel_files[0],
                    'file2': excel_files[1]
                })
    
    return folder_pairs

def find_best_key_columns(df1, df2, matches_df):
    """Находит лучшие ключевые колонки для соединения"""
    
    # Ищем сопоставленные колонки с высокой уникальностью
    matched_pairs = matches_df[matches_df['status'].isin(['MATCHED', 'PREDEFINED_KEY'])]
    
    key_candidates = []
    
    for _, match in matched_pairs.iterrows():
        col_a = match['system_a_column']
        col_b = match['system_b_column']
        
        if col_a in df1.columns and col_b in df2.columns:
            # Конвертируем в строки для проверки уникальности
            unique_pct_a = (df1[col_a].apply(lambda x: str(x).strip() if not pd.isna(x) else "").nunique() / len(df1)) * 100
            unique_pct_b = (df2[col_b].apply(lambda x: str(x).strip() if not pd.isna(x) else "").nunique() / len(df2)) * 100
            
            # Предпочтение колонкам с высокой уникальностью и хорошим score
            if unique_pct_a > 80 and unique_pct_b > 80:  # Снизил порог с 90% до 80%
                score = (unique_pct_a + unique_pct_b + match['combined_score']) / 3
                key_candidates.append((col_a, col_b, score))
                print(f"Кандидат в ключи: '{col_a}' ({unique_pct_a:.1f}%) -> '{col_b}' ({unique_pct_b:.1f}%) score: {score:.1f}")
    
    if key_candidates:
        # Выбираем лучшую пару по комбинированному score
        best_candidate = max(key_candidates, key=lambda x: x[2])
        return best_candidate[0], best_candidate[1]
    
    # Если не нашли, используем первую predefined или matched пару
    if not matched_pairs.empty:
        first_match = matched_pairs.iloc[0]
        print(f"Используем первую сопоставленную пару: '{first_match['system_a_column']}' -> '{first_match['system_b_column']}'")
        return first_match['system_a_column'], first_match['system_b_column']
    
    # Если совсем ничего не нашли, пробуем найти по отдельности
    key_col_a = find_best_single_key_column(df1)
    key_col_b = find_best_single_key_column(df2)
    
    if key_col_a and key_col_b:
        print(f"Используем отдельно найденные ключи: '{key_col_a}' -> '{key_col_b}'")
        return key_col_a, key_col_b
    
    return None, None


def find_best_single_key_column(df):
    """Находит лучшую ключевую колонку в одном DataFrame"""
    best_col = None
    best_score = 0
    
    for col in df.columns:
        # Конвертируем в строки для проверки уникальности
        unique_vals = df[col].apply(lambda x: str(x).strip() if not pd.isna(x) else "")
        unique_pct = (unique_vals.nunique() / len(df)) * 100
        null_pct = (df[col].isnull().sum() / len(df)) * 100
        
        # Score учитывает уникальность и отсутствие null
        score = unique_pct * (100 - null_pct) / 100
        
        if score > best_score and unique_pct > 80 and null_pct < 5:
            best_score = score
            best_col = col
    
    return best_col

def create_summary_report(all_summary_data, output_path):
    """Создает сводный отчет по всем папкам с полной статистикой"""
    
    if not all_summary_data:
        print("Нет данных для сводного отчета")
        return
    
    # Создаем DataFrame с полной статистикой
    summary_records = []
    for stats in all_summary_data:
        summary_records.append({
            'Папка': stats['folder_name'],
            'Всего строк': stats['total_rows'],
            'Строк с расхождениями': stats['rows_with_mismatches'],
            'Строк без расхождений': stats['rows_without_mismatches'],
            '% строк с расхождениями': f"{(stats['rows_with_mismatches']/stats['total_rows'])*100:.2f}%" if stats['total_rows'] > 0 else "0%",
            'Колонок система A': stats['total_columns_a'],
            'Колонок система B': stats['total_columns_b'],
            'Сопоставленных колонок': stats['matched_columns'],
            'Колонок с расхождениями': stats['columns_with_mismatches'],
            'Всего расхождений': stats['total_mismatches'],
            '% совпадения': f"{stats['match_percentage']:.2f}%",
            'Только в системе A': stats.get('left_only_rows', 0),
            'Только в системе B': stats.get('right_only_rows', 0),
            'В обеих системах': stats.get('both_sides_rows', 0)
        })
    
    summary_df = pd.DataFrame(summary_records)
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Сводный отчет по всем папкам"
    
    # Записываем заголовки
    headers = list(summary_df.columns)
    for col_idx, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col_idx, value=header)
        cell.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
        cell.font = Font(bold=True, color="FFFFFF")
        cell.alignment = Alignment(horizontal='center')
    
    # Записываем данные
    for row_idx, (_, row) in enumerate(summary_df.iterrows(), 2):
        for col_idx, value in enumerate(row, 1):
            ws.cell(row=row_idx, column=col_idx, value=value)
    
    # Добавляем итоги
    total_row = len(summary_df) + 2
    ws.cell(row=total_row, column=1, value="ИТОГО:").font = Font(bold=True)
    
    # Вычисляем итоги
    if not summary_df.empty:
        ws.cell(row=total_row, column=2, value=summary_df['Всего строк'].sum())
        ws.cell(row=total_row, column=3, value=summary_df['Строк с расхождениями'].sum())
        ws.cell(row=total_row, column=4, value=summary_df['Строк без расхождений'].sum())
        ws.cell(row=total_row, column=5, value=f"{(summary_df['Строк с расхождениями'].sum()/summary_df['Всего строк'].sum())*100:.2f}%")
        ws.cell(row=total_row, column=6, value=summary_df['Колонок система A'].sum())
        ws.cell(row=total_row, column=7, value=summary_df['Колонок система B'].sum())
        ws.cell(row=total_row, column=8, value=summary_df['Сопоставленных колонок'].sum())
        ws.cell(row=total_row, column=9, value=summary_df['Колонок с расхождениями'].sum())
        ws.cell(row=total_row, column=10, value=summary_df['Всего расхождений'].sum())
        
        # Средний процент совпадения
        avg_match = summary_df['% совпадения'].str.rstrip('%').astype(float).mean()
        ws.cell(row=total_row, column=11, value=f"{avg_match:.2f}%")
        
        ws.cell(row=total_row, column=12, value=summary_df['Только в системе A'].sum())
        ws.cell(row=total_row, column=13, value=summary_df['Только в системе B'].sum())
        ws.cell(row=total_row, column=14, value=summary_df['В обеих системах'].sum())
    
    # Настраиваем ширину колонок
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    output_file = output_path / "summary_report_all_folders.xlsx"
    wb.save(output_file)
    print(f"\nСводный отчет сохранен: {output_file}")
    
    # Выводим сводную статистику в консоль
    print("\n=== СВОДНАЯ СТАТИСТИКА ПО ВСЕМ ПАПКАМ ===")
    for stats in all_summary_data:
        print(f"\nПапка: {stats['folder_name']}")
        print(f"  Строк с расхождениями: {stats['rows_with_mismatches']}/{stats['total_rows']} ({stats['rows_with_mismatches']/stats['total_rows']*100:.1f}%)")
        print(f"  Всего расхождений: {stats['total_mismatches']}")
        print(f"  Процент совпадения: {stats['match_percentage']:.1f}%")


In [17]:
def get_column_name_by_index(df, column_index):
    """Получает название колонки по индексу (начиная с 0)"""
    if column_index < len(df.columns):
        return df.columns[column_index]
    return None

def process_all_folders_with_reports(base_folder_path, output_base_path, 
                                   predefined_key_columns=None, predefined_key_indices=None):
    """Обрабатывает все папки и создает отчеты"""
    
    folder_pairs = analyze_folder_structure(base_folder_path)
    
    if not folder_pairs:
        print("Не найдено пар файлов для сравнения!")
        return
    
    output_path = Path(output_base_path)
    output_path.mkdir(exist_ok=True)
    
    comparator = AdvancedColumnComparator(similarity_threshold=75)
    all_summary_data = []
    
    for pair in folder_pairs:
        print(f"\n{'='*50}")
        print(f"Обработка папки: {pair['folder']}")
        print(f"{'='*50}")
        
        try:
            # Чтение файлов
            df1 = pd.read_excel(pair['file1'])
            df2 = pd.read_excel(pair['file2'])
            
            print(f"Система A: {len(df1)} строк, {len(df1.columns)} колонок")
            print("Колонки системы A:")
            for i, col in enumerate(df1.columns):
                print(f"  [{i}] {col}")
                
            print(f"Система B: {len(df2)} строк, {len(df2.columns)} колонок") 
            print("Колонки системы B:")
            for i, col in enumerate(df2.columns):
                print(f"  [{i}] {col}")
            
            # Обрабатываем predefined ключевые колонки по индексам
            final_predefined_keys = None
            if predefined_key_indices:
                key_idx_a, key_idx_b = predefined_key_indices
                key_col_a = get_column_name_by_index(df1, key_idx_a)
                key_col_b = get_column_name_by_index(df2, key_idx_b)
                
                if key_col_a and key_col_b:
                    final_predefined_keys = (key_col_a, key_col_b)
                    print(f"Используются ключевые колонки по индексам: [{key_idx_a}] {key_col_a} -> [{key_idx_b}] {key_col_b}")
                else:
                    print(f"Ошибка: неверные индексы колонок {predefined_key_indices}")
            
            # Если не заданы по индексам, используем predefined по именам
            if not final_predefined_keys and predefined_key_columns:
                final_predefined_keys = predefined_key_columns
                print(f"Используются ключевые колонки по именам: {predefined_key_columns}")
            
            # Поиск соответствий с predefined ключевыми колонками
            matches_df, profiles1, profiles2 = comparator.find_column_matches(
                df1, df2, 
                predefined_key_columns=final_predefined_keys
            )
            
            print("\nРезультаты сопоставления колонок:")
            print(matches_df.to_string())
            
            # Автоматически находим ключевые колонки
            key_col_a, key_col_b = find_best_key_columns(df1, df2, matches_df)
            
            if key_col_a and key_col_b:
                print(f"Ключевые колонки для сравнения: {key_col_a} -> {key_col_b}")
                
                # Создаем детальный отчет
                summary_stats = comparator.create_detailed_mismatch_report(
                    df1, df2, matches_df, (key_col_a, key_col_b), 
                    pair['folder'], output_path
                )
                
                all_summary_data.append(summary_stats)
                
                # Выводим краткую статистику
                matched_count = len(matches_df[matches_df['status'].isin(['MATCHED', 'PREDEFINED_KEY'])])
                print(f"Сопоставлено колонок: {matched_count}")
                print(f"Строк с расхождениями: {summary_stats['rows_with_mismatches']}")
                print(f"Всего расхождений: {summary_stats['total_mismatches']}")
                print(f"Общий процент совпадения: {summary_stats['match_percentage']:.2f}%")
            else:
                print("Не удалось найти подходящие ключевые колонки для сравнения")
                # Показываем какие колонки есть в matches_df для отладки
                print("Доступные сопоставленные колонки:")
                matched_pairs = matches_df[matches_df['status'].isin(['MATCHED', 'PREDEFINED_KEY'])]
                for _, match in matched_pairs.iterrows():
                    print(f"  {match['system_a_column']} -> {match['system_b_column']} (score: {match['combined_score']:.1f})")
                
        except Exception as e:
            print(f"Ошибка при обработке {pair['folder']}: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # Создаем сводный отчет по всем папкам
    if all_summary_data:
        create_summary_report(all_summary_data, output_path)
    
    return all_summary_data



1. БАЗОВОЕ ИСПОЛЬЗОВАНИЕ (автоматическое определение ключей):
   results = process_all_folders_with_reports(
       base_folder_path="путь/к/папке/с/данными",
       output_base_path="путь/к/папке/для/отчетов"
   )

2. С PREDEFINED КЛЮЧЕВЫМИ КОЛОНКАМИ:
   results = process_all_folders_with_reports(
       base_folder_path="путь/к/папке/с/данными",
       output_base_path="путь/к/папке/для/отчетов",
       predefined_key_columns=('ID_из_системы_A', 'ID_из_системы_B')
   )

3. СТРУКТУРА ПАПОК:
   data_folder/
   ├── folder1/
   │   ├── file1.xlsx
   │   └── file2.xlsx
   ├── folder2/
   │   ├── data_a.xlsx
   │   └── data_b.xlsx

4. ОСОБЕННОСТИ:
   - Все данные конвертируются в строки для избежания проблем с типами
   - Predefined ключевые колонки имеют приоритет при сопоставлении
   - Создаются детальные отчеты для каждой папки и сводный отчет
   - Расхождения подсвечиваются розовым цветом
"""

In [ ]:
# ИНСТРУКЦИЯ ПО ИСПОЛЬЗОВАНИЮ:

"""
1. БАЗОВОЕ ИСПОЛЬЗОВАНИЕ (автоматическое определение ключей):
   results = process_all_folders_with_reports(
       base_folder_path="путь/к/папке/с/данными",
       output_base_path="путь/к/папке/для/отчетов"
   )

2. С PREDEFINED КЛЮЧЕВЫМИ КОЛОНКАМИ ПО ИМЕНАМ:
   results = process_all_folders_with_reports(
       base_folder_path="путь/к/папке/с/данными",
       output_base_path="путь/к/папке/для/отчетов",
       predefined_key_columns=('ID_из_системы_A', 'ID_из_системы_B')
   )

3. С PREDEFINED КЛЮЧЕВЫМИ КОЛОНКАМИ ПО НОМЕРАМ (начиная с 0):
   results = process_all_folders_with_reports(
       base_folder_path="путь/к/папке/с/данными", 
       output_base_path="путь/к/папке/для/отчетов",
       predefined_key_indices=(0, 1)  # 0-я колонка из файла A, 1-я из файла B
   )

4. ПРИМЕРЫ:

   # По номерам колонок
   results = process_all_folders_with_reports(
       "C:/data/comparison", 
       "C:/data/reports",
       predefined_key_indices=(0, 0)  # Обе первые колонки
   )

   # По именам колонок
   results = process_all_folders_with_reports(
       "C:/data/comparison",
       "C:/data/reports",
       predefined_key_columns=('ProductID', 'ItemCode')
   )



## Запуск

In [19]:
results = process_all_folders_with_reports(
       base_folder_path="comparison_data",
       output_base_path="comparison_data"
   )


Обработка папки: ликард
Система A: 595083 строк, 7 колонок
Колонки системы A:
  [0] CODE
  [1] SHORT_NAME/ru_RU
  [2] FULL_NAME/ru_RU
  [3] MSEHI
  [4] SIP_CODE
  [5] ACTUAL_TAX_VALUE_NAME
  [6] NETTO_VALUE
Система B: 374169 строк, 11 колонок
Колонки системы B:
  [0] CODE
  [1] SHORT_NAME/ru_RU
  [2] FULL_NAME/ru_RU
  [3] INT_CODE
  [4] MSEHI
  [5] ACTUAL_TAX_VALUE_NAME
  [6] NETTO_VALUE
  [7] SIP_CODE
  [8] SPPE_GOT_PROD_ID
  [9] Unnamed: 9
  [10] Unnamed: 10

Результаты сопоставления колонок:
         system_a_column        system_b_column  name_similarity  data_similarity  combined_score     status
0                   CODE                   CODE              100             75.0            85.0    MATCHED
1       SHORT_NAME/ru_RU       SHORT_NAME/ru_RU              100             75.0            85.0    MATCHED
2        FULL_NAME/ru_RU               NO_MATCH                0              0.0             0.0  UNMATCHED
3                  MSEHI                  MSEHI              10

In [18]:
results = process_all_folders_with_reports(
       "comparison_data", 
       "comparison_data",
       predefined_key_columns=('CSCD_ID', 'CSCD_ID')  # Обе первые колонки
   )


Обработка папки: ликард
Система A: 595083 строк, 7 колонок
Колонки системы A:
  [0] CODE
  [1] SHORT_NAME/ru_RU
  [2] FULL_NAME/ru_RU
  [3] MSEHI
  [4] SIP_CODE
  [5] ACTUAL_TAX_VALUE_NAME
  [6] NETTO_VALUE
Система B: 374169 строк, 11 колонок
Колонки системы B:
  [0] CODE
  [1] SHORT_NAME/ru_RU
  [2] FULL_NAME/ru_RU
  [3] INT_CODE
  [4] MSEHI
  [5] ACTUAL_TAX_VALUE_NAME
  [6] NETTO_VALUE
  [7] SIP_CODE
  [8] SPPE_GOT_PROD_ID
  [9] Unnamed: 9
  [10] Unnamed: 10
Используются ключевые колонки по именам: ('CSCD_ID', 'CSCD_ID')

Результаты сопоставления колонок:
         system_a_column        system_b_column  name_similarity  data_similarity  combined_score     status
0                   CODE                   CODE              100             75.0            85.0    MATCHED
1       SHORT_NAME/ru_RU       SHORT_NAME/ru_RU              100             75.0            85.0    MATCHED
2        FULL_NAME/ru_RU               NO_MATCH                0              0.0             0.0  UNMATCHED